In [ ]:
################################################################################
################################################################################
#
# FILE: reviews_test.ipynb 
#
# BY: Dmitry Sedov 
#
# CREATED: Thu Apr 9 2020
#
# DESC: This code tests the get_records and review_request code.
#
# EXEC: 
#      
################################################################################
################################################################################

In [ ]:
################################## Libraries ###################################

import importlib
import get_records
import review_request
import sqlalchemy as db
import pandas as pd
import psycopg2
importlib.reload(review_request)

################################################################################

In [ ]:
api_key = '***REMOVED***'
n_records = 25
log_file_path = '/home/user/projects/urban/code/data-construction/yelp-reviews/yelp_review_requests.log'
postfix = 'testing'

In [ ]:
engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname1}')
connection = engine.connect()
conn = psycopg2.connect('dbname=dataname1 user={user} password={user_pass}')

In [ ]:
searcher = review_request.Yelp_Lookup(api_key, match_type = 'y_id')
requests_table = get_records.get_request_reviews(n_records, engine, conn, postfix)

In [ ]:
requests_table

In [ ]:
result = requests_table.apply(lambda row: review_request.request_id(searcher, row), axis = 1)

In [ ]:
result = pd.concat(result.values)

In [ ]:
result

In [ ]:
success = result.loc[result['review_request_status'] == 'success'].copy()

In [ ]:
success

In [ ]:
success['time_created'] = pd.to_datetime(success['time_created'])

In [ ]:
success.drop(['review_request_status', 'review_error_message'], axis = 1, inplace = True)

In [ ]:
success.reset_index(drop = True, inplace = True)

In [ ]:
success.to_sql('reviews_yelp',
            con = engine,
            index = False,
            if_exists = 'append')

In [ ]:
engine.dispose()

In [ ]:
status_update = result.copy()
status_update.drop_duplicates(subset = 'y_id', keep = 'first', inplace = True)

In [ ]:
status_update.shape

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute(review_request.create_status_update_table_statement)

In [ ]:
rows = zip(status_update.y_id, status_update.review_request_status, status_update.review_error_message)

In [ ]:
cur.executemany(review_request.insert_status_update_table_statement, rows)

In [ ]:
cur.execute(review_request.update_review_requests_statement)
conn.commit()
conn.close()